In [54]:
#imports
import time
import qcportal as ptl
import numpy as np
import matplotlib.pyplot as plt
from fragmenter.utils import HARTREE_2_KJMOL
from simtk import unit
import os, sys
from fragmenter import chemi
import cmiles
import qcportal as ptl
from openforcefield.topology import Molecule, Topology
from openforcefield.typing.engines.smirnoff import ForceField


In [116]:
#functions 
def torsion_barrier_for_molecule(tdr_object, smiles, show_plots=False):
    """
    Takes in a single torsion drive record that has energies from multiple conformers (at different torsion angles), evaluates the torsion barrier 
    
    Parameters
    ----------
    tdr_object : object
        torsion drive record from QC archive for a molecule
    
    Returns
    -------
    tdr_object.id : int
        id of the TD record
    dihedral_indices: list
        list of atom indices for which torsion is driven in this record
    torsion_barrier: float
        torsion barrier energy in KJ/mol, maximum of all the barriers
    mol: oemol object
        oemol from the smiles in dataframe index
    """
    energies = list(tdr_object.get_final_energies().values())
    tmp = list(tdr_object.get_final_energies().keys())
    angles = [i[0]*np.pi/180 for i in tmp]
    angles, energies = zip(*sorted(zip(angles, energies)))
    angles = np.array(angles)
    energies = np.array(energies)
    angles = np.append(angles[-3:] - 2*np.pi, np.append(angles, angles[:3] + 2*np.pi))
    energies = np.append(energies[-3:], np.append(energies, energies[:3]))
    
    idx = []
    for i in range(len(angles)-2):
        m1 = (energies[i+1]-energies[i])/(angles[i+1]-angles[i])
        m2 = (energies[i+2]-energies[i+1])/(angles[i+2]-angles[i+1])
        if np.sign(m1) == np.sign(m2):
            continue
        else:
            idx.append(i+1)
        
    if (show_plots):
        min_ener = min(energies)
        energies_y = (energies-min_ener)*HARTREE_2_KJMOL
        fontsize = 14
        plt.figure()
        plt.plot(angles*180/np.pi, energies_y, 'b-X', angles[idx]*180/np.pi, energies_y[idx], 'ro')
        plt.legend(['QM data', 'Max, min'], bbox_to_anchor=(1, 1), fontsize=fontsize)
        plt.title('Torsion drive interpolation', fontsize=fontsize)
        plt.xlabel('Dihedral Angle [Degrees]', fontsize=fontsize)
        plt.ylabel('Relative energy [KJ / mol]', fontsize=fontsize)
        plt.xticks(fontsize=fontsize)
        plt.yticks(fontsize=fontsize)
        fig_name = 'plot_' + tdr_object.id + '.png'
        plt.savefig(fig_name)
        plt.show()
        
    torsion_barriers = []
    for i in range(int(len(idx)-1)):
        torsion_barriers.append(abs(HARTREE_2_KJMOL*abs(energies[idx[i]] - energies[idx[i+1]])))
    torsion_barriers = np.array(torsion_barriers)
    
    # get dihedral indices and pass on to get_wbo function
    dihedral_indices = list(tdr_object.dict()['keywords']['dihedrals'][0])
    mol = chemi.smiles_to_oemol(smiles)
    mol.SetData("TB", max(torsion_barriers))
    mol.SetData("TDindices", dihedral_indices)
    mol.SetData("TDid", tdr_object.id)
    mol.SetData("cmiles", smiles)
    return mol
    #return(tdr_object.id, dihedral_indices, max(torsion_barriers), mol)


def analyzeTorsion(tID, ff, k1, k2, qcaDS, fileName):
        """
        Description:
        This function takes in a torsion id, and performs analysis on it using a QCA torsion drive dataset
        This function will generate a plot with a .pdf extension

        Input:
        tID: String of a torsion ID from an .offxml format force field e.g. "t43"
        ff: file name of a .offxml style force field
        qcaDS: String of a name of a QCA torsion drive dataset
        fileName: String for the .pdf file name of the plot of WBO versus torsion drive barrier
        k1: Float of the force constant for wiberg bond order equal to 1 for the specified torsion
        k2: Float of the force constant for wiberg bond order equal to 2 for the specified torsion

        Return:
        resultDict: A dictionary of the results from the residual calculation. The key is the smiles and the
        items is a list of the [torsion barrier hieght, WBO, residual].
        """

        #WIP Load in the QCA dataset, convert the dataset into OEMols with data tags containing WBO and torsion barrier height
        molList = loadDataset(qcaDS)


        molsTorsion=[]
        for mol in molList:
            status = checkTorsion(tID, ff, mol)
            if status == True:
                molsTorsion.append(mol)

        resultDict={}

        for m in molsTorsion:
            resultDict[OEMolsToSmiles(m)]= [m.GetData("TorsionBarrier"), m.GetData("WBO"), getResidual(m, k1, k2)]

        plotResults(resultDict, fileName, k1, k2)


        return resultDict

def loadDataset(datasets):
    """
        To load torsion drive datasets and return a list of molecules with torsion barriers and associated wiberg bond order
        
        Parameters
        ----------
        datasets : list of tuples containing the dataset name and specification
        
        Returns
        -------
        molList : each row contains the tdr_object.id, oemol_object, dihedral_indices, torsion_barrier, wbo
    """
    molList = []
    for dataset_tuple in datasets:
        print("Reading dataset: ", dataset_tuple[0], ", and specification: ", dataset_tuple[1])
        molList.extend(loadDataset_low(dataset_tuple[0], dataset_tuple[1]))

    return molList
    

def loadDataset_low(datasetName, specification):
    """
    Low level call to load each torsion drive dataset and return a list of molecules
    
        Parameters
        ----------
        datasetName : str
            torsion drive dataset name.  
        specification : str
            specification in the dataset. Example: "B3LYP-D3", "default", "UFF" 
        
        Returns
        -------
        molList : list of objects
            each row contains the tdr_object.id, dihedral_indices, torsion_barrier, oemol_object
    """
    while True:    
        try:
            assert(datasetName)
            break
        except AssertionError:
            print ("datasetName is empty. Check input list of dataset tuples")
            raise
    while True:    
        try:
            assert(specification)
            break
        except AssertionError:
            print ("specification is empty. Check input list of dataset tuples")
            raise

    # initiate qc portal instance
    client = ptl.FractalClient()    
    # from the TorsionDriveDataset collection picking up given datasetName
    ds = client.get_collection("TorsionDriveDataset", datasetName)
    ds.status([specification], status="COMPLETE")
 
    # Serial implementation
    tb = []
    for i in range(ds.df.size):
        if (ds.df.iloc[i,0].status == 'COMPLETE'):
            smiles = ds.df.index[i]
            cmiles_ids = cmiles.get_molecule_ids(smiles, strict=False)
            mapped_smiles = cmiles_ids['canonical_isomeric_explicit_hydrogen_mapped_smiles']
            tb.append(torsion_barrier_for_molecule(ds.df.iloc[i, 0], mapped_smiles))
    print("Total records processed for this dataset:", len(tb), "out of ", len(ds.df))
    return(tb)



def getResidual(oemol, k1, k2):
    """
    Description:
    Finds the residual of an oemol by taking the input k1 and k2 values and finding the y=mx+b form and calculates
    the difference from the wbo, torsion barrier height point

    input:
    oemol: OEMol from oechem, contains datatags for WBO and torsion barrier height
    k1: Float of the force constant for wiberg bond order equal to 1 for the specified torsion
    k2: Float of the force constant for wiberg bond order equal to 2 for the specified torsion
    """
    #WIP

    return residual

def plotResult(resultDict, filename, k1, k2):
    """
    Description:
    Creates a scatter plot of WBO versus the torsion barrier height with the y=mx + b form of the torsion interpolation.
    Saves plot to .pdf

    Input:
    resultDict: Dictionary of the data for plotting. The keys are smiles, and the items are a list of WBO, torsion barrier height
    and residual
    fileName: Name of file to save .pdf form of the plot
    k1: Float of the force constant for wiberg bond order equal to 1 for the specified torsion
    k2: Float of the force constant for wiberg bond order equal to 2 for the specified torsion

    return:
    """
    #WIP
    return


def checkTorsion(molList, TID):
    """
    definition: takem ollist and check if the molecules in a list match a specific torsion id
    """
    matches=[]
    count=0
    for mol in molList: 
        molecule = Molecule.from_mapped_smiles(mol.GetData("cmiles"))
        topology = Topology.from_molecules([molecule])
        
        # Let's label using the Parsley force field
        forcefield = ForceField('openff_unconstrained-1.2.1.offxml')
        # Run the molecule labeling
        molecule_force_list = forcefield.label_molecules(topology)
        params=[]
        # Print out a formatted description of the torsion parameters applied to this molecule
        for mol_idx, mol_forces in enumerate(molecule_force_list):
            #print(f'Forces for molecule {mol_idx}')
            for force_tag, force_dict in mol_forces.items():
                if force_tag == 'ProperTorsions':
                    #print(f"\n{force_tag}:")                    
                    for (atom_indices, parameter) in force_dict.items():
                        atomList=[]
                        for idx in atom_indices:
                            atomList.append(idx)
                        #print(atomList)
                        #print(mol.GetData("TDindices"))
                        params.append(parameter.id)
                        #print(atomList)
                        #print(atomList[::-1])
                        if atomList == mol.GetData("TDindices") or atomList[::-1] == mol.GetData("TDindices"):
                            count+=1
                            print(type(parameter.id))
                            print(parameter.id)
                            if parameter.id == TID:
                                print("Match!")
                                matches.append(mol)
                        #print("atoms: %s  parameter_id: %s  smirks %s" % (atomstr, parameter.id, parameter.smirks) )
    print(count)
    print(len(molList))
    return matches, params


In [117]:
#datasetName="OpenFF Gen 2 Torsion Set 5 Bayer"
datasetName="OpenFF Rowley Biaryl v1.0"
molList=loadDataset_low(datasetName, "default")
mols=checkTorsion(molList, 't43')
print(mols)





RDKit ERROR: [10:54:40] Can't kekulize mol.  Unkekulized atoms: 6 9 10 11 12 13 15
RDKit ERROR: 
RDKit ERROR: [10:54:40] Can't kekulize mol.  Unkekulized atoms: 6 9 10 11 12 13 15
RDKit ERROR: 
RDKit ERROR: [10:54:40] Can't kekulize mol.  Unkekulized atoms: 6 9 10 11 12 13 15
RDKit ERROR: 
RDKit ERROR: [10:54:40] Can't kekulize mol.  Unkekulized atoms: 6 9 10 11 12 13 15
RDKit ERROR: 
RDKit ERROR: [10:54:40] Can't kekulize mol.  Unkekulized atoms: 6 9 10 11 12 13 15
RDKit ERROR: 


Total records processed for this dataset: 86 out of  87
[0, 1, 6, 11]
[11, 6, 1, 0]
[0, 1, 6, 18]
[18, 6, 1, 0]
[0, 4, 9, 8]
[8, 9, 4, 0]
[0, 4, 9, 11]
[11, 9, 4, 0]
[1, 0, 4, 9]
[9, 4, 0, 1]
[1, 0, 4, 16]
[16, 4, 0, 1]
[1, 6, 11, 9]
[9, 11, 6, 1]
[2, 3, 8, 7]
[7, 8, 3, 2]
[2, 3, 8, 9]
[9, 8, 3, 2]
[2, 5, 10, 7]
[7, 10, 5, 2]
[3, 2, 5, 10]
[10, 5, 2, 3]
[3, 2, 5, 17]
[17, 5, 2, 3]
[3, 8, 7, 10]
[10, 7, 8, 3]
[3, 8, 7, 19]
[19, 7, 8, 3]
[3, 8, 9, 4]
[4, 9, 8, 3]
[3, 8, 9, 11]
[11, 9, 8, 3]
<class 'str'>
t43
Match!
[4, 0, 1, 6]
[6, 1, 0, 4]
[4, 0, 1, 13]
[13, 1, 0, 4]
[4, 9, 8, 7]
[7, 8, 9, 4]
[4, 9, 11, 6]
[6, 11, 9, 4]
[5, 2, 3, 8]
[8, 3, 2, 5]
[5, 2, 3, 15]
[15, 3, 2, 5]
[5, 10, 7, 8]
[8, 7, 10, 5]
[5, 10, 7, 19]
[19, 7, 10, 5]
[6, 1, 0, 12]
[12, 0, 1, 6]
[6, 11, 9, 8]
[8, 9, 11, 6]
[7, 8, 3, 15]
[15, 3, 8, 7]
[7, 8, 9, 11]
[11, 9, 8, 7]
[7, 10, 5, 17]
[17, 5, 10, 7]
[8, 3, 2, 14]
[14, 2, 3, 8]
[8, 9, 4, 16]
[16, 4, 9, 8]
[9, 4, 0, 12]
[12, 0, 4, 9]
[9, 8, 3, 15]
[15, 3, 8, 9]
[9, 8, 

[0, 1, 6, 7]
[7, 6, 1, 0]
[0, 1, 6, 10]
[10, 6, 1, 0]
[0, 5, 10, 6]
[6, 10, 5, 0]
[1, 0, 5, 10]
[10, 5, 0, 1]
[1, 0, 5, 16]
[16, 5, 0, 1]
[1, 6, 7, 8]
[8, 7, 6, 1]
[1, 6, 7, 9]
[9, 7, 6, 1]
[1, 6, 10, 5]
[5, 10, 6, 1]
[2, 3, 8, 7]
[7, 8, 3, 2]
[2, 4, 9, 7]
[7, 9, 4, 2]
[3, 2, 4, 9]
[9, 4, 2, 3]
[3, 2, 4, 15]
[15, 4, 2, 3]
[3, 8, 7, 6]
[6, 7, 8, 3]
[3, 8, 7, 9]
[9, 7, 8, 3]
[4, 2, 3, 8]
[8, 3, 2, 4]
[4, 2, 3, 14]
[14, 3, 2, 4]
[4, 9, 7, 6]
[6, 7, 9, 4]
[4, 9, 7, 8]
[8, 7, 9, 4]
[5, 0, 1, 6]
[6, 1, 0, 5]
[5, 0, 1, 12]
[12, 1, 0, 5]
[5, 10, 6, 7]
[7, 6, 10, 5]
[6, 1, 0, 11]
[11, 0, 1, 6]
[6, 10, 5, 16]
[16, 5, 10, 6]
[7, 6, 1, 12]
[12, 1, 6, 7]
[7, 8, 3, 14]
[14, 3, 8, 7]
[7, 9, 4, 15]
[15, 4, 9, 7]
[8, 3, 2, 13]
[13, 2, 3, 8]
[8, 7, 6, 10]
[10, 6, 7, 8]
[9, 4, 2, 13]
[13, 2, 4, 9]
[9, 7, 6, 10]
[10, 6, 7, 9]
[10, 5, 0, 11]
[11, 0, 5, 10]
[10, 6, 1, 12]
[12, 1, 6, 10]
[11, 0, 1, 12]
[12, 1, 0, 11]
[11, 0, 5, 16]
[16, 5, 0, 11]
[13, 2, 3, 14]
[14, 3, 2, 13]
[13, 2, 4, 15]
[15, 4, 2, 13]
[0

[0, 1, 4, 8]
[8, 4, 1, 0]
[0, 1, 4, 16]
[16, 4, 1, 0]
[0, 2, 5, 8]
[8, 5, 2, 0]
[0, 2, 5, 17]
[17, 5, 2, 0]
[1, 0, 2, 5]
[5, 2, 0, 1]
[1, 0, 2, 14]
[14, 2, 0, 1]
[1, 4, 8, 5]
[5, 8, 4, 1]
[1, 4, 8, 9]
[9, 8, 4, 1]
[2, 0, 1, 4]
[4, 1, 0, 2]
[2, 0, 1, 13]
[13, 1, 0, 2]
[2, 5, 8, 4]
[4, 8, 5, 2]
[2, 5, 8, 9]
[9, 8, 5, 2]
[3, 6, 9, 8]
[8, 9, 6, 3]
[3, 6, 9, 11]
[11, 9, 6, 3]
[3, 7, 11, 9]
[9, 11, 7, 3]
[3, 7, 11, 10]
[10, 11, 7, 3]
[4, 1, 0, 12]
[12, 0, 1, 4]
[4, 8, 5, 17]
[17, 5, 8, 4]
[4, 8, 9, 6]
[6, 9, 8, 4]
[4, 8, 9, 11]
[11, 9, 8, 4]
[5, 2, 0, 12]
[12, 0, 2, 5]
[5, 8, 4, 16]
[16, 4, 8, 5]
[5, 8, 9, 6]
[6, 9, 8, 5]
[5, 8, 9, 11]
[11, 9, 8, 5]
[6, 3, 7, 11]
[11, 7, 3, 6]
[6, 3, 7, 19]
[19, 7, 3, 6]
[6, 9, 11, 7]
[7, 11, 9, 6]
[6, 9, 11, 10]
[10, 11, 9, 6]
[7, 3, 6, 9]
[9, 6, 3, 7]
[7, 3, 6, 18]
[18, 6, 3, 7]
[7, 11, 9, 8]
[8, 9, 11, 7]
[7, 11, 10, 20]
[20, 10, 11, 7]
[7, 11, 10, 21]
[21, 10, 11, 7]
[7, 11, 10, 22]
[22, 10, 11, 7]
[8, 4, 1, 13]
[13, 1, 4, 8]
[8, 5, 2, 14]
[14, 2, 5, 8]


[0, 1, 3, 7]
[7, 3, 1, 0]
[0, 1, 3, 15]
[15, 3, 1, 0]
[0, 2, 4, 7]
[7, 4, 2, 0]
[0, 2, 4, 16]
[16, 4, 2, 0]
[1, 0, 2, 4]
[4, 2, 0, 1]
[1, 0, 2, 14]
[14, 2, 0, 1]
[1, 3, 7, 4]
[4, 7, 3, 1]
[1, 3, 7, 10]
[10, 7, 3, 1]
[2, 0, 1, 3]
[3, 1, 0, 2]
[2, 0, 1, 13]
[13, 1, 0, 2]
[2, 4, 7, 3]
[3, 7, 4, 2]
[2, 4, 7, 10]
[10, 7, 4, 2]
[3, 1, 0, 12]
[12, 0, 1, 3]
[3, 7, 4, 16]
[16, 4, 7, 3]
[3, 7, 10, 8]
[8, 10, 7, 3]
[3, 7, 10, 9]
[9, 10, 7, 3]
[4, 2, 0, 12]
[12, 0, 2, 4]
[4, 7, 3, 15]
[15, 3, 7, 4]
[4, 7, 10, 8]
[8, 10, 7, 4]
[4, 7, 10, 9]
[9, 10, 7, 4]
[5, 6, 9, 10]
[10, 9, 6, 5]
[5, 6, 9, 19]
[19, 9, 6, 5]
[5, 8, 10, 7]
[7, 10, 8, 5]
[5, 8, 10, 9]
[9, 10, 8, 5]
[6, 5, 8, 10]
[10, 8, 5, 6]
[6, 5, 8, 11]
[11, 8, 5, 6]
[6, 9, 10, 7]
[7, 10, 9, 6]
[6, 9, 10, 8]
[8, 10, 9, 6]
[7, 3, 1, 13]
[13, 1, 3, 7]
[7, 4, 2, 14]
[14, 2, 4, 7]
[7, 10, 8, 11]
[11, 8, 10, 7]
[7, 10, 9, 19]
[19, 9, 10, 7]
[8, 5, 6, 9]
[9, 6, 5, 8]
[8, 5, 6, 18]
[18, 6, 5, 8]
[8, 10, 9, 19]
[19, 9, 10, 8]
[9, 6, 5, 17]
[17, 5, 6, 9]


...

[0, 1, 5, 9]
[9, 5, 1, 0]
[0, 1, 5, 20]
[20, 5, 1, 0]
[0, 2, 6, 9]
[9, 6, 2, 0]
[0, 2, 6, 21]
[21, 6, 2, 0]
[1, 0, 2, 6]
[6, 2, 0, 1]
[1, 0, 2, 17]
[17, 2, 0, 1]
[1, 5, 9, 6]
[6, 9, 5, 1]
[1, 5, 9, 12]
[12, 9, 5, 1]
[2, 0, 1, 5]
[5, 1, 0, 2]
[2, 0, 1, 16]
[16, 1, 0, 2]
[2, 6, 9, 5]
[5, 9, 6, 2]
[2, 6, 9, 12]
[12, 9, 6, 2]
[3, 4, 8, 11]
[11, 8, 4, 3]
[3, 4, 8, 23]
[23, 8, 4, 3]
[3, 7, 10, 11]
[11, 10, 7, 3]
[3, 7, 10, 13]
[13, 10, 7, 3]
[4, 3, 7, 10]
[10, 7, 3, 4]
[4, 3, 7, 22]
[22, 7, 3, 4]
[4, 8, 11, 10]
[10, 11, 8, 4]
[4, 8, 11, 14]
[14, 11, 8, 4]
[5, 1, 0, 15]
[15, 0, 1, 5]
[5, 9, 6, 21]
[21, 6, 9, 5]
[5, 9, 12, 13]
[13, 12, 9, 5]
[5, 9, 12, 14]
[14, 12, 9, 5]
<class 'str'>
t47
[6, 2, 0, 15]
[15, 0, 2, 6]
[6, 9, 5, 20]
[20, 5, 9, 6]
[6, 9, 12, 13]
[13, 12, 9, 6]
[6, 9, 12, 14]
[14, 12, 9, 6]
[7, 3, 4, 8]
[8, 4, 3, 7]
[7, 3, 4, 19]
[19, 4, 3, 7]
[7, 10, 11, 8]
[8, 11, 10, 7]
[7, 10, 11, 14]
[14, 11, 10, 7]
[7, 10, 13, 12]
[12, 13, 10, 7]
[8, 4, 3, 18]
[18, 3, 4, 8]
[8, 11, 10, 13]
[1

[0, 1, 3, 7]
[7, 3, 1, 0]
[0, 1, 3, 14]
[14, 3, 1, 0]
[0, 2, 4, 7]
[7, 4, 2, 0]
[0, 2, 4, 15]
[15, 4, 2, 0]
[1, 0, 2, 4]
[4, 2, 0, 1]
[1, 0, 2, 13]
[13, 2, 0, 1]
[1, 3, 7, 4]
[4, 7, 3, 1]
[1, 3, 7, 8]
[8, 7, 3, 1]
[2, 0, 1, 3]
[3, 1, 0, 2]
[2, 0, 1, 12]
[12, 1, 0, 2]
[2, 4, 7, 3]
[3, 7, 4, 2]
[2, 4, 7, 8]
[8, 7, 4, 2]
[3, 1, 0, 11]
[11, 0, 1, 3]
[3, 7, 4, 15]
[15, 4, 7, 3]
[3, 7, 8, 5]
[5, 8, 7, 3]
[3, 7, 8, 9]
[9, 8, 7, 3]
[4, 2, 0, 11]
[11, 0, 2, 4]
[4, 7, 3, 14]
[14, 3, 7, 4]
[4, 7, 8, 5]
[5, 8, 7, 4]
[4, 7, 8, 9]
[9, 8, 7, 4]
[5, 6, 10, 9]
[9, 10, 6, 5]
[5, 6, 10, 18]
[18, 10, 6, 5]
[5, 8, 9, 10]
[10, 9, 8, 5]
[6, 5, 8, 7]
[7, 8, 5, 6]
[6, 5, 8, 9]
[9, 8, 5, 6]
[6, 10, 9, 8]
[8, 9, 10, 6]
[7, 3, 1, 12]
[12, 1, 3, 7]
[7, 4, 2, 13]
[13, 2, 4, 7]
[7, 8, 5, 16]
[16, 5, 8, 7]
[7, 8, 9, 10]
[10, 9, 8, 7]
[8, 5, 6, 10]
[10, 6, 5, 8]
[8, 5, 6, 17]
[17, 6, 5, 8]
[8, 7, 3, 14]
[14, 3, 7, 8]
[8, 7, 4, 15]
[15, 4, 7, 8]
[8, 9, 10, 18]
[18, 10, 9, 8]
[9, 8, 5, 16]
[16, 5, 8, 9]
[9, 10, 6, 17]
[

[0, 1, 7, 5]
[5, 7, 1, 0]
[0, 2, 8, 5]
[5, 8, 2, 0]
[1, 0, 2, 8]
[8, 2, 0, 1]
[1, 0, 2, 13]
[13, 2, 0, 1]
[1, 7, 5, 6]
[6, 5, 7, 1]
[1, 7, 5, 8]
[8, 5, 7, 1]
[2, 0, 1, 7]
[7, 1, 0, 2]
[2, 0, 1, 12]
[12, 1, 0, 2]
[2, 8, 5, 6]
[6, 5, 8, 2]
[2, 8, 5, 7]
[7, 5, 8, 2]
[3, 4, 10, 6]
[6, 10, 4, 3]
[3, 4, 10, 16]
[16, 10, 4, 3]
[3, 9, 6, 5]
[5, 6, 9, 3]
[3, 9, 6, 10]
[10, 6, 9, 3]
[4, 3, 9, 6]
[6, 9, 3, 4]
[4, 10, 6, 5]
[5, 6, 10, 4]
[4, 10, 6, 9]
[9, 6, 10, 4]
[5, 6, 10, 16]
[16, 10, 6, 5]
[5, 7, 1, 12]
[12, 1, 7, 5]
[5, 8, 2, 13]
[13, 2, 8, 5]
[6, 9, 3, 14]
[14, 3, 9, 6]
[6, 10, 4, 15]
[15, 4, 10, 6]
[7, 1, 0, 11]
[11, 0, 1, 7]
[7, 5, 6, 9]
[9, 6, 5, 7]
[7, 5, 6, 10]
[10, 6, 5, 7]
[8, 2, 0, 11]
[11, 0, 2, 8]
[8, 5, 6, 9]
[9, 6, 5, 8]
[8, 5, 6, 10]
[10, 6, 5, 8]
[9, 3, 4, 10]
[10, 4, 3, 9]
[9, 3, 4, 15]
[15, 4, 3, 9]
[9, 6, 10, 16]
[16, 10, 6, 9]
[10, 4, 3, 14]
[14, 3, 4, 10]
[11, 0, 1, 12]
[12, 1, 0, 11]
[11, 0, 2, 13]
[13, 2, 0, 11]
[14, 3, 4, 15]
[15, 4, 3, 14]
[15, 4, 10, 16]
[16, 10, 4, 

[0, 2, 6, 7]
[7, 6, 2, 0]
[0, 2, 6, 9]
[9, 6, 2, 0]
[0, 4, 8, 9]
[9, 8, 4, 0]
[0, 4, 8, 10]
[10, 8, 4, 0]
[1, 3, 7, 6]
[6, 7, 3, 1]
[1, 3, 7, 11]
[11, 7, 3, 1]
[1, 5, 11, 7]
[7, 11, 5, 1]
[2, 0, 4, 8]
[8, 4, 0, 2]
[2, 0, 4, 16]
[16, 4, 0, 2]
[2, 6, 7, 3]
[3, 7, 6, 2]
[2, 6, 7, 11]
[11, 7, 6, 2]
[2, 6, 9, 8]
[8, 9, 6, 2]
[2, 6, 9, 18]
[18, 9, 6, 2]
[3, 1, 5, 11]
[11, 5, 1, 3]
[3, 1, 5, 17]
[17, 5, 1, 3]
[3, 7, 6, 9]
[9, 6, 7, 3]
[3, 7, 11, 5]
[5, 11, 7, 3]
[4, 0, 2, 6]
[6, 2, 0, 4]
[4, 0, 2, 14]
[14, 2, 0, 4]
[4, 8, 9, 6]
[6, 9, 8, 4]
[4, 8, 9, 18]
[18, 9, 8, 4]
[5, 1, 3, 7]
[7, 3, 1, 5]
[5, 1, 3, 15]
[15, 3, 1, 5]
[5, 11, 7, 6]
[6, 7, 11, 5]
[6, 2, 0, 12]
[12, 0, 2, 6]
[6, 7, 3, 15]
[15, 3, 7, 6]
[6, 9, 8, 10]
[10, 8, 9, 6]
[7, 3, 1, 13]
[13, 1, 3, 7]
[7, 6, 2, 14]
[14, 2, 6, 7]
[7, 6, 9, 8]
[8, 9, 6, 7]
[7, 6, 9, 18]
[18, 9, 6, 7]
[7, 11, 5, 17]
[17, 5, 11, 7]
[8, 4, 0, 12]
[12, 0, 4, 8]
[9, 6, 2, 14]
[14, 2, 6, 9]
[9, 6, 7, 11]
[11, 7, 6, 9]
<class 'str'>
t43
Match!
[9, 8, 4, 16]
[16

[0, 1, 6, 10]
[10, 6, 1, 0]
[0, 1, 6, 17]
[17, 6, 1, 0]
[0, 5, 10, 6]
[6, 10, 5, 0]
[0, 5, 10, 7]
[7, 10, 5, 0]
[1, 0, 5, 10]
[10, 5, 0, 1]
[1, 0, 5, 16]
[16, 5, 0, 1]
[1, 6, 10, 5]
[5, 10, 6, 1]
[1, 6, 10, 7]
[7, 10, 6, 1]
[2, 3, 8, 7]
[7, 8, 3, 2]
[2, 4, 9, 7]
[7, 9, 4, 2]
[3, 2, 4, 9]
[9, 4, 2, 3]
[3, 2, 4, 15]
[15, 4, 2, 3]
[3, 8, 7, 9]
[9, 7, 8, 3]
[3, 8, 7, 10]
[10, 7, 8, 3]
[4, 2, 3, 8]
[8, 3, 2, 4]
[4, 2, 3, 14]
[14, 3, 2, 4]
[4, 9, 7, 8]
[8, 7, 9, 4]
[4, 9, 7, 10]
[10, 7, 9, 4]
[5, 0, 1, 6]
[6, 1, 0, 5]
[5, 0, 1, 12]
[12, 1, 0, 5]
[5, 10, 6, 17]
[17, 6, 10, 5]
[5, 10, 7, 8]
[8, 7, 10, 5]
[5, 10, 7, 9]
[9, 7, 10, 5]
[6, 1, 0, 11]
[11, 0, 1, 6]
[6, 10, 5, 16]
[16, 5, 10, 6]
[6, 10, 7, 8]
[8, 7, 10, 6]
[6, 10, 7, 9]
[9, 7, 10, 6]
[7, 8, 3, 14]
[14, 3, 8, 7]
[7, 9, 4, 15]
[15, 4, 9, 7]
[7, 10, 5, 16]
[16, 5, 10, 7]
[7, 10, 6, 17]
[17, 6, 10, 7]
[8, 3, 2, 13]
[13, 2, 3, 8]
[9, 4, 2, 13]
[13, 2, 4, 9]
[10, 5, 0, 11]
[11, 0, 5, 10]
[10, 6, 1, 12]
[12, 1, 6, 10]
[11, 0, 1, 12]
[12, 1,

In [ ]:
import qcportal as ptl
from openforcefield.topology import Molecule, Topology
from openforcefield.typing.engines.smirnoff import ForceField


client = ptl.FractalClient()
ds = client.get_collection("TorsionDriveDataset", "OpenFF Fragmenter Phenyl Benchmark")
ds.status("B3LYP-D3", status="COMPLETE")
# output = torsion_barrier_for_molecule(ds.df.iloc[1, 0], ds.df.index[1], True)
# print("Torsionderive record Id, Dihedral_indices, Torsion barrier, oemol")
# print(output)
dihedral_indices = list(ds.df.iloc[1, 0].dict()['keywords']['dihedrals'][0])
print("Dihedral indices for which torsion is driven: ", dihedral_indices)
molecule = Molecule.from_smiles(ds.df.index[1])
display(molecule.visualize)
topology = Topology.from_molecules([molecule])
# Let's label using the Parsley force field
forcefield = ForceField('openff-1.2.1.offxml')
# Run the molecule labeling
molecule_force_list = forcefield.label_molecules(topology)
# Print out a formatted description of the torsion parameters applied to this molecule
for mol_idx, mol_forces in enumerate(molecule_force_list):
    print(f'Forces for molecule {mol_idx}')
    for force_tag, force_dict in mol_forces.items():
        if force_tag == 'ProperTorsions':
            print(f"\n{force_tag}:")
            for (atom_indices, parameter) in force_dict.items():
                atomstr=''
                for idx in atom_indices:
                    atomstr += '%3s' % idx
                print("atoms: %s  parameter_id: %s  smirks %s" % (atomstr, parameter.id, parameter.smirks) )